In [ ]:
import pandas as pd

pd_siniestros = pd.read_excel('../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx')
excel_path = '../data/raw/base_anuario_de_siniestralidad_2024_Final.xlsx'

In [3]:
###
# 1. Ver todas las hojas
###
print('Hojas disponibles en el archivo:')
hojas = pd.ExcelFile(excel_path).sheet_names
for i, hoja in enumerate(hojas):
    print(f"   {i+1}. {hoja}")

Hojas disponibles en el archivo:
   1. Siniestros
   2. Vehiculos
   3. Actor_vial
   4. Limpieza de datos
   5. Diccionario


In [5]:
###
# 2. Filtrar hojas que no quieres (apuntes, tareas)
###
hojas_interes = []
for hoja in hojas:
    if 'siniestros' in hoja.lower() or 'limpieza' in hoja.lower() or 'diccionario' in hoja.lower():
        continue
    else:
        hojas_interes.append(hoja)

print(f'\nHojas a analizar: {hojas_interes}')


Hojas a analizar: ['Vehiculos', 'Actor_vial']


In [ ]:
###
# 3. Ver todas las columnas de cada hoja de interes (función)
###
def analizar_hoja_detallada(hoja_nombre):
    '''Función para análisis detallado de cada hoja'''
    separador = '=' * 60
    print(f'\n{separador}')
    print(f'Analisis detallado - hoja: {hoja_nombre}')
    print(f'{separador}')
    
    # Cargar la hoja
    df_sinistros_clean = pd.read_excel(excel_path, sheet_name=hoja_nombre)
    
    # Información básica
    print(f'Dimensiones: {df_sinistros_clean.shape[0]} filas × {df_sinistros_clean.shape[1]} columnas')
    
    # Columnas
    print(f'\nLista de Columnas ({len(df_sinistros_clean.columns)} total):')
    for i, columna in enumerate(df_sinistros_clean.columns, 1):
        print(f'   {i:2d}. {columna}')
    
    # Tipos de datos
    print(f'\nTipo de dátos:')
    print(df_sinistros_clean.dtypes)
    
    # Valores nulos
    print(f'\nValores nulos (primeras 10 columnas):')
    nulos = df_sinistros_clean.isnull().sum()
    print(nulos)
    
    return df_sinistros_clean



In [ ]:
###
# 3. Ver todas las columnas de cada hoja de interes (ejecución)
###
dataframes = {}
for hoja in hojas_interes:
    dataframes[hoja] = analizar_hoja_detallada(hoja)


Analisis detallado - hoja: Vehiculos
Dimensiones: 523168 filas × 32 columnas

Lista de Columnas (32 total):
    1. Codigo_Accidente
    2. Formulario
    3. Fecha_Acc
    4. AA_Acc
    5. Codigo_Vehiculo
    6. Clase
    7. Servicio
    8. Modalidad
    9. Vehiculo_Viajaba_Clasificado
   10. Tipo_SITP
   11. Con_Bicicleta
   12. Con_Carga
   13. Con_Embriaguez
   14. Con_Huecos
   15. Con_Menores
   16. Con_Moto
   17. Con_Peaton
   18. Con_Persona_Mayor
   19. Con_Rutas
   20. Con_Tpi
   21. Con_Tpp
   22. Con_Velocidad
   23. Con_Sitp
   24. Con_Troncal
   25. Con_Alimentador
   26. Con_Zonal
   27. Con_Provisional
   28. Con_Articulado
   29. Con_Biarticulado
   30. Con_Padron_Dual
   31. Con_Servicio_Especial
   32. Con_Taxi

Tipo de dátos:
Codigo_Accidente                         int64
Formulario                              object
Fecha_Acc                       datetime64[ns]
AA_Acc                                   int64
Codigo_Vehiculo                        float64
Clase    

In [ ]:
###
# 4. Tabla de analisis para valores unicos vs nulos (función)
###
def generar_tabla_analisis(hoja_nombre):
    """Genera solo la tabla de análisis sin resumen"""
    df = dataframes[hoja_nombre]
    
    separador = '=' * 80
    print(f'\n{separador}')
    print(f'Tabla de Análisis - Hoja: {hoja_nombre}')
    print(f'{separador}')
    
    resultados = []
    
    for columna in df.columns:
        unique_vals = df[columna].nunique()
        total_vals = df.shape[0]
        nulos = df[columna].isnull().sum()
        
        pct_nulos = round((nulos / total_vals * 100), 2) if total_vals > 0 else 0
        pct_unicos = round((unique_vals / total_vals * 100), 2) if total_vals > 0 else 0
        tipo_dato = df[columna].dtype
        
        # Evaluar utilidad
        if pct_nulos > 80:
            utilidad = 'ELIMINAR'
        elif unique_vals == 1:
            utilidad = 'ELIMINAR'
        elif pct_unicos > 90:
            utilidad = 'REVISAR'
        else:
            utilidad = 'MANTENER'
        
        resultados.append({
            'Columna': columna,
            'Tipo': tipo_dato,
            'Unicos': unique_vals,
            '% Unicos': pct_unicos,
            'Nulos': nulos,
            '% Nulos': pct_nulos,
            'Utilidad': utilidad
        })
    
    df_resultados = pd.DataFrame(resultados)
    print(df_resultados.to_string(index=False))
    
    return df_resultados



In [ ]:
###
# 4. Tabla de analisis para valores unicos vs nulos (ejecución)
###
for hoja in hojas_interes:
    tabla_resultados = generar_tabla_analisis(hoja)


Tabla de Análisis - Hoja: Vehiculos
                     Columna           Tipo  Unicos  % Unicos  Nulos  % Nulos Utilidad
            Codigo_Accidente          int64  278466     53.23      0     0.00 MANTENER
                  Formulario         object  278466     53.23      0     0.00 MANTENER
                   Fecha_Acc datetime64[ns]    3653      0.70      0     0.00 MANTENER
                      AA_Acc          int64      10      0.00      0     0.00 MANTENER
             Codigo_Vehiculo        float64      30      0.01      3     0.00 MANTENER
                       Clase         object      22      0.00   4023     0.77 MANTENER
                    Servicio         object       5      0.00      0     0.00 MANTENER
                   Modalidad         object       8      0.00 421229    80.52 ELIMINAR
Vehiculo_Viajaba_Clasificado         object       8      0.00      0     0.00 MANTENER
                   Tipo_SITP         object       8      0.00 475607    90.91 ELIMINAR
      

In [ ]:
###
# Filtrado de eliminar y revisar (ejecución)
for hoja in hojas_interes:
    tabla_resultados = generar_tabla_analisis(hoja)
    
    # Mostrar solo las problemáticas
    problematicas = tabla_resultados[tabla_resultados['Utilidad'] != 'MANTENER']
    if not problematicas.empty:
        print(f'\n--- Columnas problemáticas en {hoja} ---')
        print(problematicas.to_string(index=False))


Tabla de Análisis - Hoja: Vehiculos
                     Columna           Tipo  Unicos  % Unicos  Nulos  % Nulos Utilidad
            Codigo_Accidente          int64  278466     53.23      0     0.00 MANTENER
                  Formulario         object  278466     53.23      0     0.00 MANTENER
                   Fecha_Acc datetime64[ns]    3653      0.70      0     0.00 MANTENER
                      AA_Acc          int64      10      0.00      0     0.00 MANTENER
             Codigo_Vehiculo        float64      30      0.01      3     0.00 MANTENER
                       Clase         object      22      0.00   4023     0.77 MANTENER
                    Servicio         object       5      0.00      0     0.00 MANTENER
                   Modalidad         object       8      0.00 421229    80.52 ELIMINAR
Vehiculo_Viajaba_Clasificado         object       8      0.00      0     0.00 MANTENER
                   Tipo_SITP         object       8      0.00 475607    90.91 ELIMINAR
      

In [63]:
import pandas as pd
excel_path = "C:\\Users\\andre\\OneDrive\\Documentos\\bootcamp analisis de datos\\analisis de datos\\proyecto_analisis_de_datos.py\\base_anuario_de_siniestralidad_2024_Final.xlsx"
archivo_excel = pd.ExcelFile(excel_path)

# Ver las hojas disponibles
print("Hojas disponibles en el archivo:")
print(archivo_excel.sheet_names)

Hojas disponibles en el archivo:
['Siniestros', 'Vehiculos', 'Actor_vial', 'Limpieza de datos', 'Diccionario']


In [64]:
#cargar la hoja 'Siniestros' es mi hoja de interes segun el analisis previo y la asignacion de tareas
df_siniestros = pd.read_excel(excel_path, sheet_name='Siniestros')
df_siniestros.head()

,Codigo_Accidente,Formulario,Longitud,Latitud,Direccion,Fecha_Acc,AA_Acc,MM_Acc,DD_Mes_Acc,Dia_Semana_Acc,...,Con_Sitp,Con_Troncal,Con_Alimentador,Con_Zonal,Con_Provisional,Con_Articulado,Con_Biarticulado,Con_Padron_Dual,Con_Servicio_Especial,Con_Taxi
0,4404210,A685,-74.181387,4.621978,AV AVENIDA CIUDAD DE CALI-KR 55 S 02,2015-02-09,2015,Febrero,9,lunes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4404211,A666,-74.027304,4.762901,AK 7-CL 186 02,2015-02-06,2015,Febrero,6,viernes,...,SI,NaN,NaN,SI,NaN,NaN,NaN,NaN,NaN,NaN
2,4404212,A698,-74.053529,4.706939,KR 45-CL 127 02,2015-02-07,2015,Febrero,7,sábado,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4404213,A000042495,-74.099563,4.576256,KR 12-CL 24 S 06,2015-02-09,2015,Febrero,9,lunes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4404214,A644,-74.091667,4.524080,CL 70-KR 11D SE 04,2015-02-08,2015,Febrero,8,domingo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
#verificar las filas y columnas disponibles
df_siniestros.shape

(278614, 40)

In [66]:
# definir nuestras columnas de interes, para posterormente segmentar la data y tomar los registros pertenecientes a 2020 a 2024
print("Columnas disponibles:")
for i, col in enumerate(df_siniestros.columns, 1):
    print(f"{i}. {col}")

Columnas disponibles:
1. Codigo_Accidente
2. Formulario
3. Longitud
4. Latitud
5. Direccion
6. Fecha_Acc
7. AA_Acc
8. MM_Acc
9. DD_Mes_Acc
10. Dia_Semana_Acc
11. Hora_Acc
12. Min_Acc
13. Localidad
14. Clase_Acc
15. Elemento_Choque
16. Tipo_Objeto_Fijo
17. Gravedad_Indicador_Tradicional
18. Gravedad_indicador_30d
19. Con_Bicicleta
20. Con_Carga
21. Con_Embriaguez
22. Con_Huecos
23. Con_Menores
24. Con_Moto
25. Con_Peaton
26. Con_Persona_Mayor
27. Con_Rutas
28. Con_Tpi
29. Con_Tpp
30. Con_Velocidad
31. Con_Sitp
32. Con_Troncal
33. Con_Alimentador
34. Con_Zonal
35. Con_Provisional
36. Con_Articulado
37. Con_Biarticulado
38. Con_Padron_Dual
39. Con_Servicio_Especial
40. Con_Taxi


In [67]:
#renombrar columna "AA_Acc" a "Año_Accidente" verificamos que el cambio se haya realizado correctamente
df_siniestros = df_siniestros.rename(columns={"AA_Acc": "Año_Accidente"})
print("Columnas disponibles:")
for i, col in enumerate(df_siniestros.columns, 1):
    print(f"{i}. {col}")


Columnas disponibles:
1. Codigo_Accidente
2. Formulario
3. Longitud
4. Latitud
5. Direccion
6. Fecha_Acc
7. Año_Accidente
8. MM_Acc
9. DD_Mes_Acc
10. Dia_Semana_Acc
11. Hora_Acc
12. Min_Acc
13. Localidad
14. Clase_Acc
15. Elemento_Choque
16. Tipo_Objeto_Fijo
17. Gravedad_Indicador_Tradicional
18. Gravedad_indicador_30d
19. Con_Bicicleta
20. Con_Carga
21. Con_Embriaguez
22. Con_Huecos
23. Con_Menores
24. Con_Moto
25. Con_Peaton
26. Con_Persona_Mayor
27. Con_Rutas
28. Con_Tpi
29. Con_Tpp
30. Con_Velocidad
31. Con_Sitp
32. Con_Troncal
33. Con_Alimentador
34. Con_Zonal
35. Con_Provisional
36. Con_Articulado
37. Con_Biarticulado
38. Con_Padron_Dual
39. Con_Servicio_Especial
40. Con_Taxi


In [68]:
#segmentamos la data para tomar los registros pertenecientes a 2020 a 2024
df_siniestros = df_siniestros[df_siniestros['Año_Accidente'].isin([2020, 2021, 2022, 2023, 2024])]
df_siniestros.shape

(105094, 40)

In [69]:
for col in df_siniestros.columns:
    print(col)

Codigo_Accidente
Formulario
Longitud
Latitud
Direccion
Fecha_Acc
Año_Accidente
MM_Acc
DD_Mes_Acc
Dia_Semana_Acc
Hora_Acc
Min_Acc
Localidad
Clase_Acc
Elemento_Choque
Tipo_Objeto_Fijo
Gravedad_Indicador_Tradicional
Gravedad_indicador_30d
Con_Bicicleta
Con_Carga
Con_Embriaguez
Con_Huecos
Con_Menores
Con_Moto
Con_Peaton
Con_Persona_Mayor
Con_Rutas
Con_Tpi
Con_Tpp
Con_Velocidad
Con_Sitp
Con_Troncal
Con_Alimentador
Con_Zonal
Con_Provisional
Con_Articulado
Con_Biarticulado
Con_Padron_Dual
Con_Servicio_Especial
Con_Taxi


In [70]:
# Identificar columnas que contienen "con_"
cols_a_eliminar = [col for col in df_siniestros.columns if "con_" in col.lower()]

print("Columnas eliminadas:")
print(cols_a_eliminar)

# Eliminar las columnas
df_siniestros = df_siniestros.drop(columns=cols_a_eliminar)

# Confirmar columnas restantes
print("\nColumnas restantes en el DataFrame:")
print(df_siniestros.columns.tolist())


Columnas eliminadas:
['Con_Bicicleta', 'Con_Carga', 'Con_Embriaguez', 'Con_Huecos', 'Con_Menores', 'Con_Moto', 'Con_Peaton', 'Con_Persona_Mayor', 'Con_Rutas', 'Con_Tpi', 'Con_Tpp', 'Con_Velocidad', 'Con_Sitp', 'Con_Troncal', 'Con_Alimentador', 'Con_Zonal', 'Con_Provisional', 'Con_Articulado', 'Con_Biarticulado', 'Con_Padron_Dual', 'Con_Servicio_Especial', 'Con_Taxi']

Columnas restantes en el DataFrame:
['Codigo_Accidente', 'Formulario', 'Longitud', 'Latitud', 'Direccion', 'Fecha_Acc', 'Año_Accidente', 'MM_Acc', 'DD_Mes_Acc', 'Dia_Semana_Acc', 'Hora_Acc', 'Min_Acc', 'Localidad', 'Clase_Acc', 'Elemento_Choque', 'Tipo_Objeto_Fijo', 'Gravedad_Indicador_Tradicional', 'Gravedad_indicador_30d']


In [71]:
# Verificar columnas restantes
for col in df_siniestros.columns:
    print(col)

Codigo_Accidente
Formulario
Longitud
Latitud
Direccion
Fecha_Acc
Año_Accidente
MM_Acc
DD_Mes_Acc
Dia_Semana_Acc
Hora_Acc
Min_Acc
Localidad
Clase_Acc
Elemento_Choque
Tipo_Objeto_Fijo
Gravedad_Indicador_Tradicional
Gravedad_indicador_30d


In [73]:
#eliminar columnas irrelevantes que no sirven para el analisis y verificamos que se hayan eliminado correctamente

df_siniestros = df_siniestros.drop(columns=['Formulario'])

# Verificar resultado
df_siniestros.head()

,Codigo_Accidente,Longitud,Latitud,Direccion,Fecha_Acc,Año_Accidente,MM_Acc,DD_Mes_Acc,Dia_Semana_Acc,Hora_Acc,Min_Acc,Localidad,Clase_Acc,Elemento_Choque,Tipo_Objeto_Fijo,Gravedad_Indicador_Tradicional,Gravedad_indicador_30d
169947,10509738,-74.125084,4.672291,KR 86-CL 24 02,2020-01-31,2020,Enero,31,viernes,10,41,FONTIBÓN,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169948,10509739,-74.051785,4.732306,CL 150-KR 48 12,2020-01-31,2020,Enero,31,viernes,21,23,SUBA,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169949,10509740,-74.125649,4.672324,CL 24-KR 86 02,2020-01-31,2020,Enero,31,viernes,10,20,FONTIBÓN,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169950,10509741,-74.135092,4.542815,AV AVENIDA BOYACA-CL 15D S 02,2020-01-31,2020,Enero,31,viernes,12,18,CIUDAD BOLÍVAR,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169951,10509742,-74.100031,4.561922,KR 4B-CL 36 S 53,2020-01-31,2020,Enero,31,viernes,1,12,SAN CRISTÓBAL,Choque,Objeto Fijo,VEHICULO,Solo Daños,Solo Daños


2 commit

In [ ]:
# renombramiento de columnas para un mejor entendimiento de las mismas, validacion
df_siniestros = df_siniestros.rename(columns={
    "Fecha_Acc": "Fecha_Accidente",
    "MM_Acc": "Mes_Accidente",
    "DD_Acc": "Día_Accidente",
    "DD_Mes_Acc": "Día_Accidente",
    "Dia_Semana_Acc": "Día_Semana_Accidente",
    "Hora_Acc": "Hora_Accidente",
    "Min_Acc": "Minuto_Accidente",
    "Clase_Acc": "Clase_Accidente",
})
df_siniestros.head()

,Codigo_Accidente,Longitud,Latitud,Direccion,Fecha_Accidente,Año_Accidente,Mes_Accidente,Día_Accidente,Día_Semana_Accidente,Hora_Accidente,Minuto_Accidente,Localidad,Clase_Accidente,Elemento_Choque,Tipo_Objeto_Fijo,Gravedad_Indicador_Tradicional,Gravedad_indicador_30d
169947,10509738,-74.125084,4.672291,KR 86-CL 24 02,2020-01-31,2020,Enero,31,viernes,10,41,FONTIBÓN,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169948,10509739,-74.051785,4.732306,CL 150-KR 48 12,2020-01-31,2020,Enero,31,viernes,21,23,SUBA,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169949,10509740,-74.125649,4.672324,CL 24-KR 86 02,2020-01-31,2020,Enero,31,viernes,10,20,FONTIBÓN,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169950,10509741,-74.135092,4.542815,AV AVENIDA BOYACA-CL 15D S 02,2020-01-31,2020,Enero,31,viernes,12,18,CIUDAD BOLÍVAR,Choque,Vehículo,NaN,Solo Daños,Solo Daños
169951,10509742,-74.100031,4.561922,KR 4B-CL 36 S 53,2020-01-31,2020,Enero,31,viernes,1,12,SAN CRISTÓBAL,Choque,Objeto Fijo,VEHICULO,Solo Daños,Solo Daños


In [ ]:
#normalizar los registros de la columna Tipo_Objeto_Fijo, validacion de cambios
df_siniestros['Tipo_Objeto_Fijo'] = df_siniestros['Tipo_Objeto_Fijo'].str.capitalize()
df_siniestros['Tipo_Objeto_Fijo'].unique()

array([nan, 'Vehiculo', 'Tren', 'Arbol', 'Semaforo', 'Objeto fijo',
       'Poste', 'Inmueble', 'Muro', 'Semoviente', 'Por identificar',
       'Baranda', 'Vehiculo estacionado', 'Valla-señal', 'Otro',
       'Tarima-caseta', 'Hidrante'], dtype=object)

In [79]:
#imputacions de datos faltantes en la columna Tipo_Objeto_Fijo con el valor "No aplica"
df_siniestros['Tipo_Objeto_Fijo'] = df_siniestros['Tipo_Objeto_Fijo'].fillna('No aplica')
df_siniestros['Tipo_Objeto_Fijo'].isnull().sum()

np.int64(0)

In [80]:
df_siniestros['Tipo_Objeto_Fijo'].unique()

array(['No aplica', 'Vehiculo', 'Tren', 'Arbol', 'Semaforo',
       'Objeto fijo', 'Poste', 'Inmueble', 'Muro', 'Semoviente',
       'Por identificar', 'Baranda', 'Vehiculo estacionado',
       'Valla-señal', 'Otro', 'Tarima-caseta', 'Hidrante'], dtype=object)

In [81]:
#imputacion de datos faltantes en la columna elemento_choque con el valor "Sin informacion"
df_siniestros['Elemento_Choque'] = df_siniestros['Elemento_Choque'].fillna('Sin informacion')
df_siniestros['Elemento_Choque'].isnull().sum()

np.int64(0)

In [82]:
df_siniestros['Elemento_Choque'].unique()

array(['Vehículo', 'Objeto Fijo', 'Sin informacion', 'Semoviente', 'Tren'],
      dtype=object)

In [83]:
#normalizar los registros de la columna Localidad, validacion de cambios
df_siniestros['Localidad'] = df_siniestros['Localidad'].str.capitalize()
df_siniestros['Localidad'].unique()

array(['Fontibón', 'Suba', 'Ciudad bolívar', 'San cristóbal', 'Bosa',
       'Kennedy', 'Usaquén', 'Chapinero', 'Engativá', 'Los mártires',
       'Barrios unidos', 'Puente aranda', 'Candelaria', 'Santa fe',
       'Teusaquillo', 'Tunjuelito', 'Antonio nariño',
       'Rafael uribe uribe', 'Usme', 'Sumapaz'], dtype=object)

In [85]:
df_siniestros.isnull().sum()

Codigo_Accidente                    0
Longitud                          288
Latitud                           288
Direccion                           0
Fecha_Accidente                     0
Año_Accidente                       0
Mes_Accidente                       0
Día_Accidente                       0
Día_Semana_Accidente                0
Hora_Accidente                      0
Minuto_Accidente                    0
Localidad                           0
Clase_Accidente                     0
Elemento_Choque                     0
Tipo_Objeto_Fijo                    0
Gravedad_Indicador_Tradicional      0
Gravedad_indicador_30d             11
dtype: int64

In [88]:
#imputacion de datos faltantes en la columna Gravedad_indicador_30 con el valor "Sin informacion"
df_siniestros['Gravedad_indicador_30d'] = df_siniestros['Gravedad_indicador_30d'].fillna('Sin informacion')
df_siniestros['Gravedad_indicador_30d'].isnull().sum()

np.int64(0)

In [89]:
df_siniestros.isnull().sum()

Codigo_Accidente                    0
Longitud                          288
Latitud                           288
Direccion                           0
Fecha_Accidente                     0
Año_Accidente                       0
Mes_Accidente                       0
Día_Accidente                       0
Día_Semana_Accidente                0
Hora_Accidente                      0
Minuto_Accidente                    0
Localidad                           0
Clase_Accidente                     0
Elemento_Choque                     0
Tipo_Objeto_Fijo                    0
Gravedad_Indicador_Tradicional      0
Gravedad_indicador_30d              0
dtype: int64